<a href="https://colab.research.google.com/github/Tadiwanashe0526/Cielo/blob/master/glam_with_BOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import gradio as gr
import PyPDF2
import time
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss
import numpy as np


In [14]:
from transformers import pipeline
qa_model = pipeline(
    "text-generation",
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",  # example variant from DeepSeek family
    device_map="auto"
)


config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.61G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/6.62G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [18]:
# --- PDF text extraction ---
def extract_text_from_pdfs(pdf_files):
    text = ""
    if pdf_files:
        for pdf in pdf_files:
            reader = PyPDF2.PdfReader(pdf)
            for page in reader.pages:
                text += page.extract_text() or ""
    return text

# --- Build vector index from PDF text ---
def build_vector_store(text):
    chunks = [text[i:i+500] for i in range(0, len(text), 500)]
    vectors = embedding_model.encode(chunks)
    index = faiss.IndexFlatL2(vectors.shape[1])
    index.add(np.array(vectors))
    return chunks, index


In [23]:
# --- Search for relevant context ---
def retrieve_answer(query, chunks, index):
    query_vector = embedding_model.encode([query])
    distances, indices = index.search(query_vector, k=2)
    context = " ".join([chunks[i] for i in indices[0]])
    return context



In [24]:

# --- Chatbot logic ---
def chatbot_response(message, pdf_files, mode, image, audio, history):
    time.sleep(1)
    if not pdf_files:
        return "", history + [(message, "Please upload at least one PDF guide first!")]

    # Extract text and build FAISS index
    pdf_text = extract_text_from_pdfs(pdf_files)
    chunks, index = build_vector_store(pdf_text)
    context = retrieve_answer(message, chunks, index)

    # Combine query with context for generation
    prompt = f"You are GlamBot, a friendly makeup assistant. Mode: {mode}. User question: {message}. Use this context:\n{context}\nAnswer clearly."

    response = qa_model(prompt, max_new_tokens=200, do_sample=True, temperature=0.7)[0]["generated_text"]

    history.append((message, response))
    return "", history

In [25]:
# --- Save chat history ---
def save_history(history):
    with open("GlamBot_chat_history.txt", "w") as f:
        for q, a in history:
            f.write(f"User: {q}\nGlamBot: {a}\n\n")
    return "💾 Chat history saved successfully!"

# --- Build Gradio UI ---
with gr.Blocks(theme=gr.themes.Soft(primary_hue="pink")) as app:
    gr.Markdown("<h1 style='text-align:center;color:#E75480;'>💄✨ GlamBot 3.0: Makeup PDF Q&A Assistant ✨💄</h1>")
    gr.Markdown("Upload your makeup guides and ask GlamBot any question! It reads the PDF and answers intelligently using an AI model.")

    with gr.Row():
        pdf_input = gr.File(label="📘 Upload Makeup Guides (PDFs)", file_count="multiple")
        image_input = gr.Image(label="📸 Upload Makeup Image (optional)", type="filepath")
        audio_input = gr.Audio(label="🎤 Upload Audio (optional)", type="filepath")

    mode = gr.Dropdown(["Makeup Tips", "Product Suggestions", "Tutorial Help"], label="Select Mode", value="Makeup Tips")
    chatbot = gr.Chatbot(label="💬 Chat with GlamBot", height=400)
    msg = gr.Textbox(label="Type your question here...")
    save_btn = gr.Button("💾 Save Chat History")

    msg.submit(chatbot_response, [msg, pdf_input, mode, image_input, audio_input, chatbot], [msg, chatbot])
    save_btn.click(save_history, [chatbot], None)

/tmp/ipython-input-1590489771.py:19: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="💬 Chat with GlamBot", height=400)


In [26]:
app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://883934da320dde5814.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
